In [1]:
#### Importing all required libraries
import requests
import pandas as pd
from bs4 import BeautifulSoup
from fake_useragent import UserAgent
ua = UserAgent()
headers = {'user-agent':ua.firefox}

base_url="http://www.pythonhow.com/real-estate/rock-springs-wy/LCWYROCKSPRINGS/t=0&s="
houses_list = []
for page in range(0,30,10):
    #### Requesting the server to get the content of the web page
    page=requests.get(base_url+str(page)+".html", headers=headers, timeout=9)
    
    #### Parsing the html
    soup=BeautifulSoup(page.content,'html.parser')
    
    #### Extracting table data using class name
    houses = soup.find_all(class_="propertyRow")
    for house in houses:
        houses_dict = {}
        
        #### Extracting 'Address' of the house
        houses_dict['Address'] = house.find_all(class_ = 'propAddressCollapse')[0].text
        
        #### Extracting 'Locality' of the house
        houses_dict["Locality"] = house.find_all(class_ = 'propAddressCollapse')[1].text
        
        #### Extracting 'Price' of the house
        houses_dict["Price"] = house.find("h4",{"class","propPrice"}).text.replace("\n","").replace(" ","")
        
        #### Extracting info of the 'Beds' in a house
        try:
            houses_dict["Beds"] = house.find(class_ = 'infoBed').find("b").text
        except:
            houses_dict["Beds"] = None
        
        #### Extracting 'Area' of the house
        try:
            houses_dict["Area"] = house.find(class_ = 'infoSqFt').find("b").text
        except:
            houses_dict["Area"] = None
        
        #### Extracting info of the 'Full Baths' in a house
        try:
            houses_dict["Full Baths"] = house.find(class_ = 'infoValueFullBath').find("b").text
        except:
            houses_dict["Full Baths"] = None
        
        #### Extracting 'info of the 'Half Baths' in a house
        try:
            houses_dict["Half Baths"] = house.find(class_ = 'infoValueHalfBath').find("b").text
        except:
            houses_dict["Half Baths"] = None
        #### Extracting 'Property Description' of the house
        try:
            houses_dict["PROPERTY DESCRIPTION"] = house.find(class_ = 'propertyDescription').text[25:-1]
        except:
            houses_dict["PROPERTY DESCRIPTION"] = None
        
        #### Extracting 'Features' of the house
        for columnGroup in house.find_all(class_ = 'columnGroup'):
            for feature_group, feature_name in zip(columnGroup.find_all(class_ = 'featureGroup'), columnGroup.find_all(class_ = 'featureName')):
                
                #### Extracting 'Lot Size' of the house
                if "Lot Size" in feature_group.text:
                    houses_dict["Lot Size"] = feature_name.text
                
                #### Extracting 'Age' of the house
                if "Age" in feature_group.text:
                    houses_dict["Age"] = feature_name.text
                
                #### Extracting 'Appliances' of the house
                if "Appliances:" in feature_group.text:
                    houses_dict["Appliances"] = feature_name.text
                
                #### Extracting 'Basement' of the house
                if "Basement" in feature_group.text:
                    houses_dict["Basement"] = feature_name.text
        
        if 'Lot Size' not in houses_dict.keys():
            houses_dict['Lot Size'] = None
        if 'Age' not in houses_dict.keys():
            houses_dict['Age'] = None
        if 'Appliances' not in houses_dict.keys():
            houses_dict['Appliances'] = None
        if 'Basement' not in houses_dict.keys():
            houses_dict['Basement'] = None
        houses_list.append(houses_dict)

#### Converting all the extracted elements into dataframe
houses_df = pd.DataFrame(houses_list) 

#### Printing top 5 rows of the data
houses_df.head(5)

,Address,Locality,Price,Beds,Area,Full Baths,Half Baths,PROPERTY DESCRIPTION,Lot Size,Age,Appliances,Basement
0,0 Gateway,"Rock Springs, WY 82901","$725,000",None,None,None,None,This Location could be used for almost anythin...,None,None,None,None
1,1003 Winchester Blvd.,"Rock Springs, WY 82901","$452,900",4,None,4,None,Beautiful new construction located on a corner...,0.21 Acres,New Construction,"Dishwasher,",Finished
2,600 Talladega,"Rock Springs, WY 82901","$396,900",5,"3,154",3,None,First Built in 2009 and upgraded this year. He...,None,6-10 Years Old,"Dishwasher,",None
3,3239 Spearhead Way,"Rock Springs, WY 82901","$389,900",4,"3,076",3,1,Beautiful Custom built home on large corner lo...,"Under 1/2 Acre,",1-5 Years Old,"Dishwasher,",Full
4,522 Emerald Street,"Rock Springs, WY 82901","$254,000",3,"1,172",3,None,Bi-level home in established neighborhood.,"Under 1/2 Acre,",31-40 Years Old,"Dishwasher,",Full


In [2]:
#### Saving the data in to csv file
houses_df.to_csv('houses.csv')